In [3]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10,6
import datetime
from pyramid.arima import auto_arima

#Read file for INDIVIDUAL PRODUCTS:
tData = iData = pd.read_table('product_distribution_training_set.txt', header=None, usecols=range(1,119))
tData.columns = range(118)
tData = tData.sum(axis=0)
iData = pd.DataFrame(iData)
iData.columns = range(118)

#Make a list of dates
dates = []
d1 = datetime.date(2018,1,1)
d2 = datetime.date(2018,4,28)
diff = d2 - d1
for i in range(diff.days + 1):
    temp = d1 + datetime.timedelta(i)
    dates.append(temp)

#Make a DataFrame from dates and data, set index to dates
df = pd.DataFrame({'Day':dates, 'TotalSale':tData})
df = df.set_index(['Day'])

rollingMean = df.rolling(window=12, min_periods=1).mean()

#Making the data stationary
df_diffscale = df.diff()#.shift(2)
tMovingAvg = df_diffscale.rolling(window=12, min_periods=1).mean()
dsdiffMinusMA = df_diffscale - tMovingAvg
#Remove the NaN values
dsdiffMinusMA.dropna(inplace=True)

#Apply auto-arima
stepwise_model = auto_arima(dsdiffMinusMA, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

stepwise_model.fit(dsdiffMinusMA)
tForecast = stepwise_model.predict(n_periods=29)
tForecast[0] = (rollingMean.iloc[117] + df.iloc[117])/2
tf = tForecast.cumsum()
tf = tf.clip(min=0)
tf = np.rint(tf).astype(int)
print(tf)

######################################################################################################################
##############################INDIVIDUAL PRODUCTS#####################################################################
#For each product, apply all the above steps
for b in range(100):
    print(b)
    iD0 = iData.iloc[b:b+1,0:118]
    iD0 = iD0.sum(axis=0)
    idf = pd.DataFrame({'Day':dates, 'TotalSale':iD0})
    idf = idf.set_index(['Day'])
    df_diffscale = idf.diff().shift(5)
    iMovingAvg = df_diffscale.rolling(window=12, min_periods=1).mean()
    dsdiffSubMA = df_diffscale - iMovingAvg
    dsdiffSubMA.dropna(inplace=True)
    stepwise_model = auto_arima(dsdiffSubMA, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=7,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)
    stepwise_model.fit(dsdiffSubMA)
    iForecast = stepwise_model.predict(n_periods=29)
    iForecast[0] = idf.iloc[117]
    ff = iForecast.cumsum()
    print("***********************")
    ff = ff.clip(min=0)
    ff = np.rint(ff).astype(int)
    print(ff)
    print("***********************")


Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1490.034, BIC=1503.256, Fit time=0.588 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1598.684, BIC=1603.973, Fit time=0.014 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=1567.056, BIC=1577.634, Fit time=0.265 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1490.302, BIC=1500.880, Fit time=0.416 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=1491.430, BIC=1507.296, Fit time=0.917 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=1528.878, BIC=1539.455, Fit time=0.154 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1490.624, BIC=1506.490, Fit time=0.770 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 1, 

***********************
[12  1  7  4  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0]
***********************
3
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=1076.313, BIC=1089.535, Fit time=0.357 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 7); AIC=1279.543, BIC=1284.832, Fit time=0.012 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 7); AIC=1181.827, BIC=1192.404, Fit time=0.172 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=1075.269, BIC=1091.135, Fit time=0.540 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 0, 7); AIC=1102.842, BIC=1116.064, Fit time=0.339 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 7); AIC=1076.820, BIC=1095.331, Fit time=0.727 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=1144.112, BIC=1154.689, Fit time=0.215 seconds
Fit ARIMA: order

Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 7); AIC=965.463, BIC=983.973, Fit time=0.869 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=999.278, BIC=1009.855, Fit time=0.085 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(2, 1, 2, 7); AIC=967.008, BIC=988.163, Fit time=1.318 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=955.020, BIC=973.531, Fit time=0.317 seconds
Fit ARIMA: order=(2, 1, 0) seasonal_order=(1, 1, 1, 7); AIC=1002.428, BIC=1018.295, Fit time=0.487 seconds
Fit ARIMA: order=(2, 1, 2) seasonal_order=(1, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 1, 7); AIC=1032.495, BIC=1045.717, Fit time=0.189 seconds
Fit ARIMA: order=(3, 1, 2) seasonal_order=(1, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=953.616, BIC=969.482

Fit ARIMA: order=(2, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=1013.488, BIC=1031.999, Fit time=0.875 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=1057.807, BIC=1071.029, Fit time=0.163 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 2, 7); AIC=1013.329, BIC=1031.839, Fit time=0.404 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(1, 1, 2, 7); AIC=1014.033, BIC=1035.188, Fit time=1.077 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=1006.522, BIC=1025.032, Fit time=0.644 seconds
Fit ARIMA: order=(3, 1, 0) seasonal_order=(0, 1, 1, 7); AIC=1053.435, BIC=1069.301, Fit time=0.202 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=1007.555, BIC=1028.710, Fit time=0.448 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=1049.729, BIC=1065.596, Fit time=0.216 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(0, 1, 2, 7); AIC=1007.161, BIC=1028.316, Fit time=0.947 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_o

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=923.774, BIC=936.996, Fit time=0.388 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 7); AIC=1053.749, BIC=1059.038, Fit time=0.011 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 7); AIC=997.771, BIC=1008.349, Fit time=0.111 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=930.809, BIC=941.386, Fit time=0.255 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=926.971, BIC=942.837, Fit time=0.330 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=965.425, BIC=976.002, Fit time=0.328 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 7); AIC=925.765, BIC=941.632, Fit time=1.011 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 7); AIC=926.726, BIC=945.237, Fit time=1.020 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=920.296, BIC=936.162, Fit time=0.671 seconds
Fit ARIMA: order=(2, 1, 0) seasonal_order=(0, 1, 1, 

...

Fit ARIMA: order=(2, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=555.318, BIC=573.829, Fit time=1.195 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=592.118, BIC=605.340, Fit time=0.439 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 2, 7); AIC=555.499, BIC=574.010, Fit time=1.757 seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(1, 1, 2, 7); AIC=557.891, BIC=579.047, Fit time=1.181 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=556.526, BIC=575.037, Fit time=0.776 seconds
Total fit time: 16.608 seconds
***********************
[3 3 4 5 5 5 5 3 3 3 4 5 4 4 2 2 2 4 4 4 4 2 2 2 3 3 3 3 1]
***********************
84
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=549.380, BIC=562.602, Fit time=0.506 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 7); AIC=721.018, BIC=726.307, Fit time=0.029 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 7); AIC=644.017, BIC=654.595, Fit time=0.195 seconds
Fit ARIMA: order=(

Fit ARIMA: order=(3, 1, 1) seasonal_order=(1, 1, 0, 7); AIC=531.828, BIC=550.339, Fit time=0.589 seconds
Total fit time: 8.933 seconds
***********************
[4 6 3 5 7 7 6 3 5 3 7 6 5 7 3 5 3 7 6 7 6 4 5 3 6 6 7 6 3]
***********************
87
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 7); AIC=714.012, BIC=719.301, Fit time=0.013 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 7); AIC=648.004, BIC=658.582, Fit time=0.089 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 0, 7); AIC=660.535, BIC=668.468, Fit time=0.034 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 0, 7); AIC=603.331, BIC=616.553, Fit time=0.208 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 1, 7); AIC=591.322, BIC=607.188, Fit time=0.251 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_ord

***********************
[1 1 0 1 2 2 1 1 0 0 1 1 2 1 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0]
***********************
91
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=522.716, BIC=535.938, Fit time=0.380 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 7); AIC=686.412, BIC=691.700, Fit time=0.011 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 7); AIC=614.106, BIC=624.684, Fit time=0.079 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=524.485, BIC=540.351, Fit time=0.671 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=575.349, BIC=585.926, Fit time=0.212 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 2, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 2, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(2, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=520.626, BIC=536.49

Fit ARIMA: order=(2, 1, 0) seasonal_order=(2, 1, 2, 7); AIC=560.753, BIC=581.908, Fit time=0.864 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(2, 1, 1, 7); AIC=511.605, BIC=535.405, Fit time=1.213 seconds
Total fit time: 13.403 seconds
***********************
[1 1 0 0 0 0 0 0 0 0 0 1 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
***********************
94
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=495.709, BIC=508.931, Fit time=0.427 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 7); AIC=672.344, BIC=677.633, Fit time=0.010 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 7); AIC=593.597, BIC=604.175, Fit time=0.065 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=497.209, BIC=513.075, Fit time=0.498 seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=528.675, BIC=539.253, Fit time=0.210 seconds
Fit ARIMA: order=(1, 1, 1) s

Fit ARIMA: order=(3, 1, 1) seasonal_order=(0, 1, 0, 7); AIC=510.991, BIC=526.858, Fit time=0.488 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(0, 1, 2, 7); AIC=467.737, BIC=488.892, Fit time=0.694 seconds
Fit ARIMA: order=(3, 1, 1) seasonal_order=(1, 1, 2, 7); AIC=469.735, BIC=493.535, Fit time=0.905 seconds
Total fit time: 9.514 seconds
***********************
[ 6  6  6  7  7  7  8  6  7  6  7  7  7  8  7  7  7  8  8  8  9  7  8  7
  8  9  9 10  8]
***********************
97
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=494.696, BIC=507.918, Fit time=0.429 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 7); AIC=683.065, BIC=688.354, Fit time=0.014 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 7); AIC=598.867, BIC=609.444, Fit time=0.067 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 7); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 7); AIC=494.423, BIC=510.290, Fit time=0.508 seconds